In [1]:
import csv
import os
import random

from tqdm import tqdm
from PIL import Image
from typing import Tuple

In [2]:
def is_square(image: Image) -> bool:
    width, height = image.size

    return height == width


def is_dataset_square(dataset_path: str) -> bool:
    files = os.listdir(dataset_path)
    incidences = list()

    for file in files:
        if file == ".DS_Store":
            continue
        with Image.open(os.path.join(dataset_path, file)) as img:
            if not is_square(img):
                incidences.append((file, img.size))
    
    if incidences:
        print(incidences)
        return False
    
    return True

In [3]:
INPUT_DATASET_PATH = "/Users/dabetm/Pictures/Sunsets/sunsets-250-square/footage"

print("Checking dims...")
assert is_dataset_square(INPUT_DATASET_PATH)

Checking dims...


In [4]:
METADATA_FILE = "sunsets_32x32.csv"


def get_average_per_channel(img: Image, n: int = None) -> Tuple[float, float, float]:
    r, g, b = 0, 0, 0

    if n:
        sample_size = min(n, img.width * img.height)
        margin = 100 if img.width > 1000 else 50
        
        for _ in range(sample_size):
            x = random.randint(0, img.width-margin)
            y = random.randint(0, img.height-margin)
            pixel = img.getpixel((x, y))

            r += pixel[0]
            g += pixel[1]
            b += pixel[2]
        
        return (r / n, g / n, b / n)

    area = img.width * img.height

    for y in range(img.height):
        for x in range(img.width):
            pixel = img.getpixel((x, y))

            r += pixel[0]
            g += pixel[1]
            b += pixel[2]
    
    if not area:
        area = 1

    return (r / area, g / area, b / area)


def generate_metadata():    
    headers = ["file", "r_avg", "g_avg", "b_avg"]
    metadata = []
    files = os.listdir(INPUT_DATASET_PATH)

    for file in tqdm(files):
        if file == ".DS_Store":
            continue
        with Image.open(os.path.join(INPUT_DATASET_PATH, file)) as img:
            r_avg, g_avg, b_avg = get_average_per_channel(img, n=500)
        filename = file.split(".")[0] + ".png"
        metadata.append(
            {
                "file": filename,
                "r_avg": r_avg,
                "g_avg": g_avg,
                "b_avg": b_avg
            }
        )
    
    with open(METADATA_FILE, "w") as output_file:
        writer = csv.DictWriter(output_file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(metadata)

In [91]:
print("Generating metadata...")
generate_metadata()

Generating metadata...


100%|██████████| 261/261 [00:22<00:00, 11.46it/s]


In [5]:
import pandas as pd


df = pd.read_csv("sunsets_16x16.csv")
df

,file,r_avg,g_avg,b_avg
0,0071.png,103.636,101.086,97.314
1,00132.png,136.842,100.248,89.894
2,00126.png,120.376,102.316,100.620
3,0065.png,93.104,64.366,105.418
4,0059.png,108.518,75.882,100.848
...,...,...,...,...
255,00103.png,108.518,94.452,110.732
256,0040.png,141.724,120.430,108.618
257,0054.png,159.838,65.040,41.398
258,00117.png,73.098,47.352,49.620


In [93]:
df.sort_values(by="r_avg")


,file,r_avg,g_avg,b_avg
21,00247.png,16.838,7.160,8.286
5,00250.png,17.574,14.352,30.138
122,00197.png,22.916,17.580,24.572
129,00196.png,25.450,18.472,23.634
94,0010.png,29.514,14.806,11.858
...,...,...,...,...
220,0084.png,204.928,150.168,95.448
36,00242.png,210.466,167.834,110.654
157,00239.png,217.724,136.278,74.082
52,00255.png,222.084,148.112,62.108


In [6]:
def format_img_filename(filename: str):
    name = filename.split(".")[0]
    return f"{name}.png"


def redim_image_dataset(dim: int, input_path, output_path):
    files = os.listdir(input_path)

    for file in tqdm(files):
        if file == ".DS_Store":
            continue
        with Image.open(os.path.join(input_path, file)) as img:
            img.thumbnail((dim, dim), Image.Resampling.LANCZOS)
            img.save(os.path.join(output_path, format_img_filename(file)), format="PNG")

In [12]:
NEW_DATASET_PATH = "/Users/dabetm/Pictures/Sunsets/sunsets-250-square/64x64"
DIM_WINDOW = 64

print("Resizing images of dataset...")
redim_image_dataset(
    DIM_WINDOW,
    input_path=INPUT_DATASET_PATH,
    output_path=NEW_DATASET_PATH
)

Resizing images of dataset...


100%|██████████| 261/261 [00:20<00:00, 12.68it/s]


In [21]:
def get_base_img(width: int = 600, height: int = 600) -> Image:
    return Image.new("RGB", (width, height), color="black")

In [103]:
get_avg_ = lambda row : (row["r_avg"] + row["g_avg"] + row["b_avg"]) / 3


df.loc[:, "avg"] = df.apply(get_avg_, axis=1)
df

,file,r_avg,g_avg,b_avg,avg
0,0071.png,103.636,101.086,97.314,100.678667
1,00132.png,136.842,100.248,89.894,108.994667
2,00126.png,120.376,102.316,100.620,107.770667
3,0065.png,93.104,64.366,105.418,87.629333
4,0059.png,108.518,75.882,100.848,95.082667
...,...,...,...,...,...
255,00103.png,108.518,94.452,110.732,104.567333
256,0040.png,141.724,120.430,108.618,123.590667
257,0054.png,159.838,65.040,41.398,88.758667
258,00117.png,73.098,47.352,49.620,56.690000


In [9]:
ASSETS_PATH = "assets"

## Create mosaic

In [13]:
from math import sqrt
from typing import List, Union


images_in_memory = dict()

def open_dataset():
    files = os.listdir(path=NEW_DATASET_PATH)

    for file in files:
        images_in_memory[file] = Image.open(os.path.join(NEW_DATASET_PATH, file))


def compute_euclidian_distance(a: List[Union[int, float]], b: List[Union[int, float]]):
    acc = 0.0

    for a_i, b_i in zip(a, b):
        acc += (a_i - b_i)**2
    
    return sqrt(acc)


def compute_nearest_images(metadata: pd.DataFrame, avg_per_channel: tuple, k: int = 3):
    distances = list() # list of tuples(distance, index)
    assert len(avg_per_channel) == 3

    distances_serie = metadata.apply(
        lambda row: (
            compute_euclidian_distance(
                a=list(avg_per_channel),
                b=[row["r_avg"], row["g_avg"], row["b_avg"]]
            ), 
            row.name
        ),
        axis=1
    )
    
    distances = list(distances_serie)
    distances.sort()

    nearest_images = list()
    for i in range(k):
        index = distances[i][1]
        nearest_images.append(metadata.iloc[index]["file"])

    return nearest_images

In [14]:
open_dataset()

k = 3

testing_images = (
    "01_sunset_and_venus.jpg",
    #"02_sunset_unidad_01.JPG",
    #"03_sunset_unidad_02.JPG",
    #"04_sunset_unidad_03.JPG",
    "05_sunset_palmeras.jpg",
    "06_sunset_gasolinera.jpg",
    "07_sunset_unidad_04.jpg",
    "08_sunset_unidad_05.jpg",
    "09_sunset_playa.png",
    "10_sunset_playa_2.png",
    "11_mona_lisa.jpg",
)

testing_images = [
    "14_noche_estrellada_4k.jpg",
]


for img_path in testing_images:
    print(img_path)
    with Image.open(os.path.join(ASSETS_PATH, img_path)) as img:
        n, m = img.height, img.width
        for y in tqdm(range(0, n, DIM_WINDOW)):
            for x in range(0, m, DIM_WINDOW):
                left = x
                upper = y
                right = min(m-1, x+DIM_WINDOW)
                lower = min(n-1, y+DIM_WINDOW)
                window = img.crop((left, upper, right, lower))

                avg = get_average_per_channel(window)
                nearest_image_names = compute_nearest_images(df, avg, k=k)
                nearest_image_name = random.choice(nearest_image_names)

                for i in enumerate(range(upper, lower)):
                    for j in enumerate(range(left, right)):
                        new_pixel = images_in_memory[nearest_image_name].getpixel(
                            (j[0], i[0])
                        )
                        img.putpixel((j[1], i[1]), value=new_pixel)

        img.save(os.path.join(ASSETS_PATH, f"out_18p_k{k}_{img_path}"))



14_noche_estrellada_4k.jpg


100%|██████████| 68/68 [01:16<00:00,  1.12s/it]


## Create collage

In [ ]:
img = get_base_img(width=924, height=1463)
#df_sorted = df.sort_values(by="r_avg")
#df_sorted = df.sort_values(by="avg")
idx = 0

df_shuffled = df.sample(frac=1)

n, m = img.height, img.width

for y in tqdm(range(0, n, DIM_WINDOW)):
    for x in range(0, m, DIM_WINDOW):
        left = x
        upper = y
        right = min(m-1, x+DIM_WINDOW)
        lower = min(n-1, y+DIM_WINDOW)
        #window = img.crop((left, upper, right, lower))

        #print(f"{idx=}")
        ref_filename = df_shuffled.iloc[idx].to_dict()["file"]
        idx += 1
        ref_img_path = f"{NEW_DATASET_PATH}/{ref_filename}"
        with Image.open(ref_img_path) as ref_img:
            for i in enumerate(range(upper, lower)):
                for j in enumerate(range(left, right)):
                    #print(j[0], i[0], ref_img.height, ref_img.width)
                    new_pixel = ref_img.getpixel(
                        (j[0], i[0])
                    )
                    img.putpixel((j[1], i[1]), value=new_pixel)
    #print("hola")


  4%|▍         | 4/92 [00:00<00:03, 28.02it/s]


IndexError: single positional indexer is out-of-bounds

In [ ]:
img.show()

In [ ]:
img.save(f"{ASSETS_PATH}/simple_collage_228_sunsets_v4.png")